In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [4]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-OTH'], ['gebaut', 'O'], ['.', 'O']]


In [36]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [37]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [38]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [39]:
print(models.label2Idx)

{'B-ORGderiv': 1, 'B-LOCpart': 12, 'I-ORGderiv': 14, 'I-LOCderiv': 2, 'I-ORGpart': 15, 'B-LOC': 3, 'B-PER': 4, 'I-OTHpart': 16, 'B-PERderiv': 17, 'B-PERpart': 18, 'B-ORGpart': 19, 'I-PER': 5, 'I-PERpart': 10, 'I-LOCpart': 20, 'B-LOCderiv': 21, 'I-ORG': 6, 'I-OTH': 22, 'B-OTHpart': 7, 'O': 23, 'B-OTH': 8, 'PADDING_TOKEN': 0, 'B-ORG': 24, 'I-LOC': 25, 'B-OTHderiv': 13, 'I-OTHderiv': 9, 'I-PERderiv': 11}


In [40]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [41]:
print(models.case2Idx)

{'mainly_numeric': 6, 'PADDING_TOKEN': 0, 'numeric': 1, 'contains_digit': 7, 'allUpper': 3, 'initialUpper': 4, 'allLower': 2, 'other': 5}


In [42]:
print(type(trainSentences))

<class 'list'>


In [43]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [44]:
models.char2Idx={"PADDING_TOKEN":0}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'2': 1, 'И': 2, 'ħ': 3, ':': 4, 'v': 5, '대': 9, 'R': 8, 'ň': 10, '-': 11, 'č': 244, 'Ш': 302, 'Ø': 12, 'ō': 13, '§': 15, 'Е': 16, 'G': 17, 'u': 247, 'Å': 18, '”': 19, '·': 281, 'J': 20, 'ж': 22, 'ź': 23, '⋅': 24, 'ý': 188, 'p': 25, '\x9a': 28, '«': 218, 'Ц': 29, '‘': 30, '9': 115, 'å': 260, 'Λ': 249, 'ê': 31, '南': 33, '!': 320, '4': 6, 'æ': 36, '懿': 37, '李': 7, '}': 38, 'ρ': 40, 'K': 41, ',': 42, '柯': 43, 'σ': 45, 'в': 46, 'H': 294, 'œ': 49, 'Þ': 48, 'р': 50, 'ῦ': 52, 'í': 53, '‐': 54, 'õ': 55, '−': 56, 'Ż': 58, 'ñ': 59, '九': 60, ']': 61, 'ă': 62, '¤': 63, 'φ': 64, 'ю': 65, '\x96': 66, '別': 67, '@': 68, 'É': 177, '$': 69, 'Ö': 283, '©': 70, 'и': 328, 'l': 71, 'г': 72, 'з': 73, 'V': 74, 'ḫ': 14, 'ç': 78, 'â': 77, 'ї': 79, 'W': 80, '鷹': 81, 'ő': 83, 'Á': 84, 'w': 86, 'd': 87, 'Ş': 89, 'ã': 233, 'ä': 90, '*': 91, 'Y': 92, 'È': 181, 'Т': 126, '+': 27, '鶴': 189, '£': 96, '守': 100, 'F': 97, 'Ä': 98, 'e': 99, '<': 101, 'с': 103, 'è': 104, 'ы': 105, 'û': 106, '`': 107, 'ą': 288, 'Î': 135, 'Č'

In [45]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [52]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [46]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [47]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


# Test Model

In [35]:
import importlib
importlib.reload(models)

<module 'models' from '/home/gwiedemann/microNER/scripts/models.py'>

In [48]:
tmp_model_filename = 'tmp_lstm3cnn_bi-lstm.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_3cnnlstm()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10560       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_22 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_24 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [50]:
importlib.reload(utils)
# print(models.max_sequence_length)

<module 'utils' from '/home/gwiedemann/microNER/scripts/utils.py'>

In [53]:
model.fit_generator(
    utils.NerSequence(trainSentences[1:100], shuffle_data=True, batch_size=32), 
    validation_data = utils.NerSequence(devSentences[1:100], batch_size=256), 
    epochs = 5, callbacks = [history]
)

Epoch 1/5
4/4 [==============================] - 17s 4s/step - loss: 1.9572 - acc: 0.5923 - val_loss: 0.4937 - val_acc: 0.9075
Epoch 2/5
4/4 [==============================] - 9s 2s/step - loss: 0.4878 - acc: 0.8939 - val_loss: 0.2820 - val_acc: 0.9623
Epoch 3/5
4/4 [==============================] - 9s 2s/step - loss: 0.2636 - acc: 0.9603 - val_loss: 0.2433 - val_acc: 0.9598
Epoch 4/5
4/4 [==============================] - 8s 2s/step - loss: 0.2360 - acc: 0.9559 - val_loss: 0.2228 - val_acc: 0.9636
Epoch 5/5
4/4 [==============================] - 8s 2s/step - loss: 0.2050 - acc: 0.9644 - val_loss: 0.2079 - val_acc: 0.9645


In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences[1:100], shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences[1:100], batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_lstm3cnn.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_lstm3cnn_bi-lstm.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)
    
    model = models.get_model_3cnnlstm()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
        validation_data = utils.NerSequence(devSentences, batch_size=256), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=512), 
        validation_data = utils.NerSequence(devSentences, batch_size=256), 
        epochs = 10, callbacks = [history]
    )
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10560       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_30 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_32 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time

47/47 [==============================] - 64s 1s/step - loss: -0.3920 - acc: 0.9951 - val_loss: -0.3830 - val_acc: 0.9916
Epoch 7/10
46/47 [============================>.] - ETA: 0s - loss: -0.3938 - acc: 0.9952
New maximum F1 score: 0.8297397769516729 (before: 0.8292230669386241) Saving to tmp_lstm3cnn_bi-lstm.0.h5
47/47 [==============================] - 64s 1s/step - loss: -0.3938 - acc: 0.9952 - val_loss: -0.3847 - val_acc: 0.9918
Epoch 8/10
47/47 [==============================] - 65s 1s/step - loss: -0.3957 - acc: 0.9953 - val_loss: -0.3863 - val_acc: 0.9917
Epoch 9/10
47/47 [==============================] - 65s 1s/step - loss: -0.3974 - acc: 0.9953 - val_loss: -0.3878 - val_acc: 0.9916
Epoch 10/10
47/47 [==============================] - 64s 1s/step - loss: -0.3992 - acc: 0.9954 - val_loss: -0.3895 - val_acc: 0.9916
Run 1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #

Epoch 3/10
46/47 [============================>.] - ETA: 0s - loss: -0.4875 - acc: 0.9952
New maximum F1 score: 0.8237901736239379 (before: 0.8222304011777696) Saving to tmp_lstm3cnn_bi-lstm.1.h5
47/47 [==============================] - 65s 1s/step - loss: -0.4875 - acc: 0.9953 - val_loss: -0.4780 - val_acc: 0.9916
Epoch 4/10
47/47 [==============================] - 66s 1s/step - loss: -0.4893 - acc: 0.9954 - val_loss: -0.4797 - val_acc: 0.9916
Epoch 5/10
47/47 [==============================] - 65s 1s/step - loss: -0.4913 - acc: 0.9955 - val_loss: -0.4810 - val_acc: 0.9915
Epoch 6/10
47/47 [==============================] - 65s 1s/step - loss: -0.4932 - acc: 0.9956 - val_loss: -0.4826 - val_acc: 0.9916
Epoch 7/10
47/47 [==============================] - 67s 1s/step - loss: -0.4950 - acc: 0.9957 - val_loss: -0.4843 - val_acc: 0.9915
Epoch 8/10
47/47 [==============================] - 65s 1s/step - loss: -0.4968 - acc: 0.9958 - val_loss: -0.4858 - val_acc: 0.9915
Epoch 9/10
46/47 [=====

Epoch 10/10
1500/1500 [==============================] - 406s 271ms/step - loss: -0.4483 - acc: 0.9936 - val_loss: -0.4673 - val_acc: 0.9914
Epoch 1/10
46/47 [============================>.] - ETA: 1s - loss: -0.4242 - acc: 0.9945
New maximum F1 score: 0.8212899648863426 (before: 0) Saving to tmp_lstm3cnn_bi-lstm.2.h5
47/47 [==============================] - 75s 2s/step - loss: -0.4242 - acc: 0.9946 - val_loss: -0.4158 - val_acc: 0.9912
Epoch 2/10
46/47 [============================>.] - ETA: 0s - loss: -0.4265 - acc: 0.9948
New maximum F1 score: 0.8223368964246222 (before: 0.8212899648863426) Saving to tmp_lstm3cnn_bi-lstm.2.h5
47/47 [==============================] - 64s 1s/step - loss: -0.4265 - acc: 0.9948 - val_loss: -0.4175 - val_acc: 0.9912
Epoch 3/10
46/47 [============================>.] - ETA: 0s - loss: -0.4284 - acc: 0.9949
New maximum F1 score: 0.8229859571322985 (before: 0.8223368964246222) Saving to tmp_lstm3cnn_bi-lstm.2.h5
47/47 [==============================] - 65s 1

1500/1500 [==============================] - 403s 268ms/step - loss: -0.2322 - acc: 0.9924 - val_loss: -0.2552 - val_acc: 0.9912
Epoch 7/10
1500/1500 [==============================] - 405s 270ms/step - loss: -0.2869 - acc: 0.9928 - val_loss: -0.3079 - val_acc: 0.9909
Epoch 8/10
1500/1500 [==============================] - 404s 270ms/step - loss: -0.3414 - acc: 0.9932 - val_loss: -0.3628 - val_acc: 0.9912
Epoch 9/10
1500/1500 [==============================] - 400s 267ms/step - loss: -0.3958 - acc: 0.9935 - val_loss: -0.4146 - val_acc: 0.9908
Epoch 10/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.4501 - acc: 0.9937
New maximum F1 score: 0.8205693296602388 (before: 0.8182156133828996) Saving to tmp_lstm3cnn_bi-lstm.3.h5
1500/1500 [==============================] - 399s 266ms/step - loss: -0.4502 - acc: 0.9937 - val_loss: -0.4687 - val_acc: 0.9912
Epoch 1/10
46/47 [============================>.] - ETA: 1s - loss: -0.4797 - acc: 0.9948
New maximum F1 score: 0.8241372

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_lstm_bi-lstm.h5', '../models/final_model-lstm_germeval_outer.h5')
with open("../models/final_model-lstm_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model-lstm_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close

In [ ]:
model = models.get_model_lstm()

model.load_weights(tmp_model_filename)

model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=512), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

true_labels, pred_labels = utils.predict_sequences(model, testSentences)

pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
f.write("\n")
f.flush()